In [7]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd

In [9]:
firefox_options = Options()
firefox_options.add_argument('--headless')

driver = webdriver.Firefox(options=firefox_options)

driver.get('https://www.mercadozapatoca.com')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [10]:
categories = soup.select('li.has-children > ul > li > a')

In [13]:
url = 'https://www.mercadozapatoca.com'
category_links = {}
for category in categories:
    category_name = category.text.strip()
    category_url = category.get('href')
    category_links[category_name] = url + category_url

In [12]:
category_links = {'DESECHABLES':'https://www.mercadozapatoca.com/c/cigarrillos_desechables/desechables/'}

In [ ]:
products = []
for category in category_links:
    print(category)
    category_url = category_links[category]
    driver.get(category_url)

    while True:
        contenedor_cat = driver.find_element(By.CLASS_NAME, 'contenedorCat')
        
        product_elements = contenedor_cat.find_elements(By.CLASS_NAME, 'dpr_container')
        
        for product in product_elements:
            try:
                product_name = product.find_element(By.CLASS_NAME, 'dpr_product-name').text.strip()
            except:
                product_name = 'No Name'
            
            try:
                product_link = product.find_element(By.CLASS_NAME, 'dpr_listname').get_attribute('href')
            except:
                product_link = 'No Link'
            
            try:
                suggested_price = product.find_element(By.CLASS_NAME, 'dpr_suggested_price').text.strip()
            except:
                suggested_price = 'No Suggested Price'
            
            try:
                list_price = product.find_element(By.CLASS_NAME, 'dpr_listprice').text.strip()
            except:
                list_price = suggested_price  # Usa el precio sugerido si no hay precio de lista
            
            try:
                discount = product.find_element(By.CLASS_NAME, 'ribbon-thumb').text.strip()
            except:
                discount = 'No Discount'
            
            try:
                in_stock = product.find_element(By.CLASS_NAME, 'dpr_in_stock').text.strip()
            except:
                in_stock = 'Out of Stock'
            
            products.append({
                'name': product_name,
                'link': product_link,
                'suggested_price': suggested_price,
                'list_price': list_price,
                'discount': discount,
                'stock': in_stock
            })
    
        try:
            next_button = driver.find_element(By.LINK_TEXT, 'Siguiente »')
            next_button.click()
            time.sleep(2) 
        except:
            break
driver.quit()

DESECHABLES
ALUMINIOS
BOLSAS
CONTENEDOR Y PORTACOMIDA
CUBIERTOS DESECHABLES
PALOS Y PALILLOS MADERA
PLATOS
VASOS Y COPAS
VINIPEL
TOALLAS DE COCINA
ACEITES
CANOLA
COCO
GIRASOL
OLIVA
SOYA
VEGETAL
ADEREZOS
PASTA DE TOMATE
SALSA
SALSA SAZONADORA
SALSAS ESPECIALES
VINAGRES
ZUMO
ARROZ


In [ ]:
df = pd.DataFrame(products)

In [ ]:
df.to_excel('productos_zapatoca.xlsx', index = False)